In [5]:
import pandas as pd
import numpy as np
from os.path import isfile
from extra_codes import calc_vif
import matplotlib.pyplot as plt
from datetime import datetime
from statsmodels.tsa.stattools import kpss
import warnings
warnings.filterwarnings("ignore")

class ML_Fraud:
    __version__='1.0.5'
    def __init__(self,sample_start=1991,test_sample=range(2001,2011),
                 OOS_per=1,OOS_gap=0,sampling='expanding',adjust_serial=True,
                 cv_type='kfold',temp_year=1,cv_flag=False,cv_k=10,write=True,IS_per=10):

        if isfile('FraudDB2020.csv')==False:
            df=pd.DataFrame()
            for s in range(1,5):
                fl_name='FraudDB2020_Part'+str(s)+'.csv'
                new_df=pd.read_csv(fl_name)
                df=df.append(new_df)
            df.to_csv('FraudDB2020.csv',index=False)
            
        df=pd.read_csv('FraudDB2020.csv')
        self.df=df
        self.ss=sample_start
        self.se=np.max(df.fyear)
        self.ts=test_sample
        self.cv_t=cv_type
        self.cv=cv_flag
        self.cv_k=cv_k
        self.cv_t_y=temp_year
        
        sampling_set=['expanding','rolling']
        if sampling in sampling_set:
            pass
        else:
            raise ValueError('Invalid sampling choice. Permitted options are "expanding" and "rolling"')
        
        self.sa=sampling
        self.w=write
        self.ip=IS_per
        self.op=OOS_per
        self.og=OOS_gap
        self.a_s=adjust_serial
        print('Module initiated successfully ...')
        #The dir() function returns all properties and methods of the specified object, without the values.
        list_methods=dir(self)
        # .any: It checks for any element satisfying a condition and returns a True in case it finds any one element.
        reduced_methods=[item+'()' for item in list_methods if any(['analy' in item,'compare' in item,item=='sumstats'])]
        #string.join(iterable)
        print('Procedures are: '+'; '.join(reduced_methods))
    
    def mc_analysis(self,B=1000,adjust_serial=None,C_FN=30,C_FP=1):
       
        from sklearn.metrics import roc_auc_score
        from extra_codes import ndcg_k,relogit
        from statsmodels.discrete.discrete_model import Logit
        from statsmodels.tools import add_constant
        
        t0=datetime.now()
        # setting the parameters
        IS_period=self.ip
        k_fold=self.cv_k
        OOS_period=self.op # 1 year ahead prediction
        OOS_gap=self.og # Gap between training and testing period
        start_OOS_year=self.ts[0] #2001
        end_OOS_year=self.ts[-1] #2010
        sample_start=self.ss #1991
        if adjust_serial==None:
            adjust_serial=self.a_s
        cross_val=self.cv
        case_window=self.sa #expanding or rolling window
        fraud_df=self.df
        write=self.w
        
        
        print('starting the MC analysis for case B='+str(B)+', serial treatment='+str(adjust_serial))
        t000=datetime.now()
        reduced_tbl_1=fraud_df.iloc[:,[0,1,3,7,8]]
        reduced_tbl_2=fraud_df.iloc[:,9:-3]
        reduced_tblset=[reduced_tbl_1,reduced_tbl_2]
        reduced_tbl=pd.concat(reduced_tblset,axis=1)
        reduced_tbl=reduced_tbl[reduced_tbl.fyear>=sample_start] #1991
        reduced_tbl=reduced_tbl[reduced_tbl.fyear<=end_OOS_year] #2010

        # Setting the cross-validation setting

        tbl_year_IS_CV=reduced_tbl.loc[np.logical_and(reduced_tbl.fyear<start_OOS_year,\
                                                   reduced_tbl.fyear>=start_OOS_year-IS_period)]
        tbl_year_IS_CV=tbl_year_IS_CV.reset_index(drop=True)
        
        X_CV=tbl_year_IS_CV.iloc[:,-11:]
        
        Y_CV=tbl_year_IS_CV.AAER_DUMMY
        
        
        X_CV_raw=tbl_year_IS_CV.iloc[:,5:-11]
        
        P_f=np.sum(Y_CV==1)/len(Y_CV)
        P_nf=1-P_f

        print('prior probablity of fraud between '+str(sample_start)+'-'+
              str(start_OOS_year-1)+' is '+str(np.round(P_f*100,2))+'%')
        
        range_oos=range(start_OOS_year,end_OOS_year+1,OOS_period) #(2001,2010+1,1)


        roc_set_ratio=np.zeros(len(range_oos))
        sensitivity_ratio=np.zeros(len(range_oos))
        specificity_ratio=np.zeros(len(range_oos))
        precision_ratio=np.zeros(len(range_oos))
        ndcg_ratio=np.zeros(len(range_oos))
        ecm_ratio=np.zeros(len(range_oos))
        
        roc_set_ratio_raw=np.zeros(len(range_oos))
        sensitivity_ratio_raw=np.zeros(len(range_oos))
        specificity_ratio_raw=np.zeros(len(range_oos))
        precision_ratio_raw=np.zeros(len(range_oos))
        ndcg_ratio_raw=np.zeros(len(range_oos))
        ecm_ratio_raw=np.zeros(len(range_oos))

        m=0
        
        for yr in range_oos: #2001-2010
            t1=datetime.now()
            if case_window=='expanding':
                year_start_IS=sample_start #1991
            else:
                year_start_IS=yr-IS_period #1991
            #how many years between training and testing sample: 
            #expanding: 1991-2000, 1991-2001
            #rolling: 1991-2000, 1992-2001
            tbl_year_IS=reduced_tbl.loc[np.logical_and(reduced_tbl.fyear<yr-OOS_gap,\
                                                       reduced_tbl.fyear>=year_start_IS)]
            tbl_year_IS=tbl_year_IS.reset_index(drop=True)  
            
            misstate_firms=np.unique(tbl_year_IS.gvkey[tbl_year_IS.AAER_DUMMY==1])
            #How many periods constitute the testing sample at a time: 2001, 2002
            tbl_year_OOS=reduced_tbl.loc[np.logical_and(reduced_tbl.fyear>=yr,\
                                                        reduced_tbl.fyear<yr+OOS_period)]
            
            if adjust_serial==True:
                ok_index=np.zeros(tbl_year_OOS.shape[0])
                for s in range(0,tbl_year_OOS.shape[0]):
                    if not tbl_year_OOS.iloc[s,1] in misstate_firms:
                        ok_index[s]=True
                    
                
            else:
                #filled with ones and keep all observations including serial frauds
                ok_index=np.ones(tbl_year_OOS.shape[0]).astype(bool)
                
            #deleting observations where a company appears both in IS and OOS samples
            tbl_year_OOS=tbl_year_OOS.iloc[ok_index==True,:]
            tbl_year_OOS=tbl_year_OOS.reset_index(drop=True)  
            
            X_train_b=tbl_year_IS.iloc[:,-11:]
            mean_vals=np.mean(X_train_b)
            std_vals=np.std(X_train_b)
            X_train_b=(X_train_b-mean_vals)/std_vals
            Y_train_b=tbl_year_IS.AAER_DUMMY
            
            X_test_b=tbl_year_OOS.iloc[:,-11:]
            X_test_b=(X_test_b-mean_vals)/std_vals
            Y_test_b=tbl_year_OOS.AAER_DUMMY
            
            X_train_raw_b=tbl_year_IS.iloc[:,5:-11]
            mean_vals_raw=np.mean(X_train_raw_b)
            std_vals_raw=np.std(X_train_raw_b)
            X_train_raw_b=(X_train_raw_b-mean_vals_raw)/std_vals_raw
            
            X_test_raw_b=tbl_year_OOS.iloc[:,5:-11]
            X_test_raw_b=(X_test_raw_b-mean_vals_raw)/std_vals_raw
            

                
            n_P=np.sum(Y_test_b==1)
            n_N=np.sum(Y_test_b==0) # how many serial frauds we have dropped from the original positive fraud cases
                
            print('1')    
            t01=datetime.now()
            #Add a column of ones to an array.
            clf_lr = Logit(Y_train_b,add_constant(X_train_b)).fit(disp=0)
            predicted_test=clf_lr.predict(add_constant(X_test_b))
            predicted_test=predicted_test.to_numpy()
            roc_set_ratio[m]=roc_auc_score(Y_test_b,predicted_test)
            #numpy.percentile()function used to compute the nth percentile of the given data (array elements) along the specified axis. 
            #classification threshold (99th percentile) = 1 or 0
            cutoff_ratio=np.percentile(predicted_test,99)
            # predicted value higher than the threshold value will flag the observation as positive, whether correctly or not (TP+FP)
            labels_ratio=(predicted_test>=cutoff_ratio).astype(int)
            #fraud correctly classified 
            sensitivity_ratio[m]=np.sum(np.logical_and(labels_ratio==1,Y_test_b==1))/np.sum(Y_test_b)
            #non-fraud correctly classified
            specificity_ratio[m]=np.sum(np.logical_and(labels_ratio==0,Y_test_b==0))/np.sum(Y_test_b==0)
            #the number of true positives to the total number of positives 
            precision_ratio[m]=np.sum(np.logical_and(labels_ratio==1,Y_test_b==1))/np.sum(labels_ratio)
            #Pandas Series.to_numpy() function is used to return a NumPy ndarray representing the values in given Series or Index.
            ndcg_ratio[m]=ndcg_k(Y_test_b.to_numpy(),predicted_test,99)
            
            FN=np.sum(np.logical_and(predicted_test<cutoff_ratio, \
                                                          Y_test_b==1))
            FP=np.sum(np.logical_and(predicted_test>=cutoff_ratio, \
                                                          Y_test_b==0))
            # C_FN: Cost of a False Negative for ECM  -30
            #C_FP: Cost of a False Positive for ECM  -1
   
            ecm_ratio[m]=C_FN*P_f*FN/n_P+C_FP*P_nf*FP/n_N
            

            clf_lr_raw = Logit(Y_train_b,add_constant(X_train_raw_b)).fit(disp=0)
            predicted_test_raw=clf_lr_raw.predict(add_constant(X_test_raw_b))
            predicted_test_raw=predicted_test_raw.to_numpy()
                
            cutoff_raw=np.percentile(predicted_test_raw,99) 
            labels_raw=(predicted_test_raw>=cutoff_raw).astype(int)
            sensitivity_ratio_raw[m]=(np.sum(np.logical_and(labels_raw==1, \
                                                         Y_test_b==1))/np.sum(Y_test_b))
            specificity_ratio_raw[m]=(np.sum(np.logical_and(labels_raw==0, \
                                                         Y_test_b==0))/np.sum(Y_test_b==0))
            precision_ratio_raw[m]=(np.sum(np.logical_and(labels_raw==1, \
                                                         Y_test_b==1))/np.sum(labels_raw))
            roc_set_ratio_raw[m]=(roc_auc_score(Y_test_b,predicted_test_raw))
            ndcg_ratio_raw[m]=ndcg_k(Y_test_b.to_numpy(),predicted_test_raw,99)
            
            FN_raw=np.sum(np.logical_and(predicted_test_raw<cutoff_raw, \
                                                          Y_test_b==1))
            FP_raw=np.sum(np.logical_and(predicted_test_raw>=cutoff_raw, \
                                                          Y_test_b==0))
            # C_FN: Cost of a False Negative for ECM  -30
            #C_FP: Cost of a False Positive for ECM  -1
   
            ecm_ratio_raw[m]=C_FN*P_f*FN_raw/n_P+C_FP*P_nf*FP_raw/n_N
         
            m+=1
         
        
        t1=datetime.now()
        dt=t1-t0
        
        f1_ratio=2*(precision_ratio*sensitivity_ratio)/(precision_ratio+sensitivity_ratio+1e-8)
        f1_ratio_raw=2*(precision_ratio_raw*sensitivity_ratio_raw)/(precision_ratio_raw+sensitivity_ratio_raw+1e-8)
        
        # create performance table now
        perf_tbl_general=pd.DataFrame()
        perf_tbl_general['models']=['Logit']       
        
        perf_tbl_general['Roc']=[str(np.round(
            np.mean(roc_set_ratio)*100,2))+'% ('+\
            str(np.round(np.std(roc_set_ratio)*100,2))+'%)']
        
        perf_tbl_general['Roc_noise_to_signal']=[str(np.round(
            np.std(roc_set_ratio)/np.mean(roc_set_ratio)*100,2))+'%']
                                                    
        perf_tbl_general['Sensitivity @ 1 Prc']=[str(np.round(
            np.mean(sensitivity_ratio)*100,2))+'% ('+\
            str(np.round(np.std(sensitivity_ratio)*100,2))+'%)']
        
        perf_tbl_general['Sensitivity_noise_to_signal']=[str(np.round(
            np.std(sensitivity_ratio)/np.mean(sensitivity_ratio)*100,2))+'%']

        perf_tbl_general['Specificity @ 1 Prc']=[str(np.round(
            np.mean(specificity_ratio)*100,2))+'% ('+\
            str(np.round(np.std(specificity_ratio)*100,2))+'%)'] 
        
        perf_tbl_general['Specificity_noise_to_signal']=[str(np.round(
            np.std(specificity_ratio)/np.mean(specificity_ratio)*100,2))+'%']
        

        perf_tbl_general['Precision @ 1 Prc']=[str(np.round(
            np.mean(precision_ratio)*100,2))+'% ('+\
            str(np.round(np.std(precision_ratio)*100,2))+'%)']
        
        perf_tbl_general['Precision_noise_to_signal']=[str(np.round(
            np.std(precision_ratio)/np.mean(precision_ratio)*100,2))+'%']

        perf_tbl_general['F1 Score @ 1 Prc']=[str(np.round(
            np.mean(f1_ratio)*100,2))+'% ('+\
            str(np.round(np.std(f1_ratio)*100,2))+'%)']
            
        perf_tbl_general['F1 Score_noise_to_signal']=[str(np.round(
            np.std(f1_ratio)/np.mean(f1_ratio)*100,2))+'%']
        
        perf_tbl_general['NDCG @ 1 Prc']=[str(np.round(
            np.mean(ndcg_ratio)*100,2))+'% ('+\
            str(np.round(np.std(ndcg_ratio)*100,2))+'%)']
        
        perf_tbl_general['NDCG_noise_to_signal']=[str(np.round(
            np.std(ndcg_ratio)/np.mean(ndcg_ratio)*100,2))+'%']
        
        
        perf_tbl_general['ECM @ 1 Prc']=[str(np.round(
            np.mean(ecm_ratio)*100,2))+'% ('+\
            str(np.round(np.std(ecm_ratio)*100,2))+'%)']
        
        perf_tbl_general['ECM_noise_to_signal']=[str(np.round(
            np.std(ecm_ratio)/np.mean(ecm_ratio)*100,2))+'%']
        
        
        
        perf_tbl_general['Roc_raw']=[str(np.round(
            np.mean(roc_set_ratio_raw)*100,2))+'% ('+\
            str(np.round(np.std(roc_set_ratio_raw)*100,2))+'%)']
        
        perf_tbl_general['Roc_noise_to_signal_raw']=[str(np.round(
            np.std(roc_set_ratio_raw)/np.mean(roc_set_ratio_raw)*100,2))+'%']
                                                    
        perf_tbl_general['Sensitivity_raw @ 1 Prc']=[str(np.round(
            np.mean(sensitivity_ratio_raw)*100,2))+'% ('+\
            str(np.round(np.std(sensitivity_ratio_raw)*100,2))+'%)']
        
        perf_tbl_general['Sensitivity_noise_to_signal_raw']=[str(np.round(
            np.std(sensitivity_ratio_raw)/np.mean(sensitivity_ratio_raw)*100,2))+'%']

        perf_tbl_general['Specificity_raw @ 1 Prc']=[str(np.round(
            np.mean(specificity_ratio_raw)*100,2))+'% ('+\
            str(np.round(np.std(specificity_ratio_raw)*100,2))+'%)'] 
        
        perf_tbl_general['Specificity_noise_to_signal_raw']=[str(np.round(
            np.std(specificity_ratio_raw)/np.mean(specificity_ratio_raw)*100,2))+'%']
        

        perf_tbl_general['Precision_raw @ 1 Prc']=[str(np.round(
            np.mean(precision_ratio_raw)*100,2))+'% ('+\
            str(np.round(np.std(precision_ratio_raw)*100,2))+'%)']
        
        perf_tbl_general['Precision_noise_to_signal_raw']=[str(np.round(
            np.std(precision_ratio_raw)/np.mean(precision_ratio_raw)*100,2))+'%']

        perf_tbl_general['F1 Score_raw @ 1 Prc']=[str(np.round(
            np.mean(f1_ratio_raw)*100,2))+'% ('+\
            str(np.round(np.std(f1_ratio_raw)*100,2))+'%)']
            
        perf_tbl_general['F1 Score_noise_to_signal_raw']=[str(np.round(
            np.std(f1_ratio_raw)/np.mean(f1_ratio_raw)*100,2))+'%']
        
        perf_tbl_general['NDCG_raw @ 1 Prc']=[str(np.round(
            np.mean(ndcg_ratio_raw)*100,2))+'% ('+\
            str(np.round(np.std(ndcg_ratio_raw)*100,2))+'%)']
        
        perf_tbl_general['NDCG_noise_to_signal_raw']=[str(np.round(
            np.std(ndcg_ratio_raw)/np.mean(ndcg_ratio_raw)*100,2))+'%']
        
        
        perf_tbl_general['ECM_raw @ 1 Prc']=[str(np.round(
            np.mean(ecm_ratio_raw)*100,2))+'% ('+\
            str(np.round(np.std(ecm_ratio_raw)*100,2))+'%)']
        
        perf_tbl_general['ECM_noise_to_signal_raw']=[str(np.round(
            np.std(ecm_ratio_raw)/np.mean(ecm_ratio_raw)*100,2))+'%']


        
        lbl_perf_tbl='MC_results_Logit'+',serial='+str(adjust_serial)+',B='+str(B)+'.csv'
                        
        if write==True:
            perf_tbl_general.to_csv(lbl_perf_tbl,index=True)
        
        t001=datetime.now()
        dt00=t001-t000
        print('MC analysis is completed after '+str(dt00.total_seconds())+' seconds')

In [6]:
a = ML_Fraud(sample_start = 1991,test_sample = range (2001,2011),OOS_per = 1,OOS_gap = 0,sampling = "expanding",adjust_serial = True,
            cv_flag = False,cv_k = 10,write = True,IS_per = 10)
a.mc_analysis()

Module initiated successfully ...
Procedures are: mc_analysis()
starting the MC analysis for case B=1000, serial treatment=True
prior probablity of fraud between 1991-2000 is 0.78%
1
1
1
1
1
1
1
1
1
1
MC analysis is completed after 4.422353 seconds
